In [31]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
import json
import pandas as pd
import warnings
from sklearn.preprocessing import RobustScaler

warnings.filterwarnings('ignore')
df = pd.read_csv("../dataset/processed/train_encoded.csv")
df["damage_grade"] = df["damage_grade"].astype('int')

s = RobustScaler()

X = df.drop(["damage_grade"], axis=1)
y = df["damage_grade"]

In [32]:
test = pd.read_csv("../dataset/processed/test_encoded.csv")
test.head()

,id,floors_before_eqtotal,old_building,plinth_areaft2,height_before_eqft,land_surface_condition,technical_solution_proposed,has_secondary_use,type_of_reinforcement_concrete,no_family_residing,...,residential_type_RentalResidential,industrial_use_type_Agro,industrial_use_type_Farm,industrial_use_type_Food,industrial_use_type_Forestbased,industrial_use_type_Infrastructure,industrial_use_type_Manufacturing,industrial_use_type_Metallurgy,industrial_use_type_Nonindustrial,industrial_use_type_ServiceTourism
0,0,2.0,-0.500000,0.308511,-0.333333,2,0,0.0,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,3.0,-0.166667,0.191489,0.833333,2,0,0.0,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2,3.0,1.333333,0.212766,0.333333,2,0,0.0,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,3,2.0,0.500000,0.095745,0.666667,2,0,0.0,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,4,2.0,-0.611111,0.079787,0.666667,2,2,0.0,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [33]:
import xgboost as xgb

params = {'colsample_bytree': 0.5, 'subsample': 0.6, 'learning_rate': 0.01,
          'n_estimators': 1029, 'max_depth': 15, 'verbosity': 2}

xgb_model = xgb.XGBClassifier(**params)
xgb_model.fit(X, y)

[23:55:48] INFO: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/tree/updater_prune.cc:98: tree pruning end, 566 extra nodes, 0 pruned nodes, max_depth=15
[23:55:48] INFO: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/tree/updater_prune.cc:98: tree pruning end, 1308 extra nodes, 0 pruned nodes, max_depth=15
[23:55:48] INFO: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/tree/updater_prune.cc:98: tree pruning end, 1346 extra nodes, 0 pruned nodes, max_depth=15
[23:55:48] INFO: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/tree/updater_prune.cc:98: tree pruning end, 1522 extra nodes, 0 pruned nodes, max_depth=15
[23:55:48] INFO: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971c

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=15, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=1029, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [40]:
import numpy as np

y_sub = xgb_model.predict(np.array(test))

submission = pd.DataFrame({
    "id": np.arange(test.shape[0]),
    "damage_grade": y_sub
})

submission.head()

,id,damage_grade
0,0,4
1,1,4
2,2,4
3,3,4
4,4,1


In [41]:
submission["damage_grade"] += 1
submission["damage_grade"].value_counts()

5    122392
3     41149
2     30568
4     27069
1     20904
Name: damage_grade, dtype: int64

In [ ]:
submission.to_csv("../dataset/submission_ohe_xgb_67.csv", index=False)